<a href="https://colab.research.google.com/github/JamieCDev20/GAM705/blob/main/705Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup

Imports

In [ ]:
!pip install pyquaternion
!pip install PyDrive

In [2]:
import numpy as np
from pyquaternion import Quaternion
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from random import random as r

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

File Reader

In [ ]:
class AnimFileReader():
  ## "#" breaks between nodes | ":" breaks between values

  def __init__(self, _path):
    self.path = _path
    self.animData = np.array(float)
    self.__ReadAnim__(self.path)

  def __ReadAnim__(self, path):
    file = open(path, 'r')
    overallData = file.read()
    overallData = overallData.split("SS")
    
    del overallData[0]

    jointCount = len(overallData)
    frameCount = len(overallData[0].split('#')[0].split(':')) - 1
    
    dataLayer = np.array(float)

    trainingData = np.array(float)

    for j in range(0, frameCount):
      for i in range(0, jointCount):

        rots = overallData[i].split('#')[0].split(':')
        poss = overallData[i].split('#')[1].split(':')

        del rots[0]
        del poss[0]

        #print(len(rots))
        #print(rots[0], rots[1], rots[2], rots[3])
        #print(dataLayer.shape)
        temp = rots[j].split(',')
        dataLayer = np.append(dataLayer, temp)
        
        temp = poss[j].split(',')
        dataLayer = np.append(dataLayer, temp)
        #print(dataLayer.shape)
      dataLayer = np.delete(dataLayer, 0)
      trainingData = np.append(trainingData, dataLayer);
      dataLayer = np.array(float)
      print("Completed: ", j + 1)

    print("Total its: ", (i + 1) * (j + 1))
    
    trainingData = np.delete(trainingData, 0)
    
    print(trainingData.shape)
    print(trainingData[0])
    
    self.animData = trainingData;
   

#File Read

In [ ]:
fr = AnimFileReader("SHO_EXORCISMWALK_1_UNCLEAN.txt")

In [ ]:
data = fr.animData
print(fr.animData.shape)
print(data.shape)
data = np.reshape(data, [221, 168])
print(fr.animData.shape)
print(data.shape)

(37128,)
(37128,)
(37128,)
(221, 168)


In [ ]:
network = Sequential(
    Input()
)

Testing

In [ ]:
a = np.array([1, 2, 3, 4, 5], float)
b = np.array([6, 6, 6, 6, 6], float)
c = b-a
b = np.append(b, c);

print(b)


[6. 6. 6. 6. 6. 5. 4. 3. 2. 1.]


#GAN Training

##Networks

Discriminator

In [12]:
discriminator = Sequential([
    Input(shape=(1)),
    Dense(5, activation='relu'),
    Dense(1)
])
discriminator.compile(optimizer="Adam", loss="mse", metrics=["Accuracy"])

Generator

In [13]:
generator = Sequential([
      Input(shape=(1)),
      Dense(5, activation='relu'),
      Dense(1)
])
generator.compile(optimizer="Adam", loss="mse", metrics=["Accuracy"])

##Training Structure

Training Vars

In [8]:
batchSize = 64
iterations = 10000
fakeDistribution = 0.5

In [6]:
def Func(x):
  return [[0.08 * (x[0]-25)**2]]

In [9]:
def GenerateBatch():
  batchDistribution = np.random.rand(batchSize)
  batchDistribution = [(0, 1)[int(x > fakeDistribution)] for x in batchDistribution]
  xTrain = [[generator.predict, Func][x] for x in batchDistribution]
  return xTrain, batchDistribution

##a = [generator.predict, Func][0]
##print(a([12]))

Training

In [16]:
for i in range(iterations):
  print("Iteration: ", i)
  discrimBatch = []
  genBatch = []
  randomNums = []
  genTrue = []
  xTrainSetup, distribution = GenerateBatch()
  
  for j in range(batchSize):
    a = r()*50
    randomNums.append(a)
    discrimBatch.append(xTrainSetup[j]([a]))
  
  discriminator.fit(np.array(discrimBatch), np.array(distribution), batchSize, verbose=0)
  results = discriminator.predict(np.array(discrimBatch))

  for j in range(batchSize):
    if distribution[j] == 0:
      genTrue.append(1-results[j])
      genBatch.append(randomNums[j])
    
  generator.fit(np.array(genBatch), np.array(genTrue), verbose=0)

Iteration:  0
Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25
Iteration:  26
Iteration:  27
Iteration:  28
Iteration:  29
Iteration:  30
Iteration:  31
Iteration:  32
Iteration:  33
Iteration:  34
Iteration:  35
Iteration:  36
Iteration:  37
Iteration:  38
Iteration:  39
Iteration:  40
Iteration:  41
Iteration:  42
Iteration:  43
Iteration:  44
Iteration:  45
Iteration:  46
Iteration:  47
Iteration:  48
Iteration:  49
Iteration:  50
Iteration:  51
Iteration:  52
Iteration:  53
Iteration:  54
Iteration:  55
Iteration:  56
Iteration:  57
Iteration:  58
Iteration:  59
Iteration:  60
Iteration:  61
Iteration:  62
Iteration:  63
Iteration:  64
Iteration:  65
Iteration:  66
Itera

KeyboardInterrupt: ignored

In [17]:
test = np.array([0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 244, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50])
print(generator.predict(test))

[[0.16484973]
 [0.22262654]
 [0.28040338]
 [0.33818018]
 [0.39595702]
 [0.45373383]
 [0.5115107 ]
 [0.56928754]
 [0.62706435]
 [0.68484116]
 [0.74261796]
 [0.80039483]
 [7.2136226 ]
 [0.91594845]
 [0.97372526]
 [1.0315021 ]
 [1.0892789 ]
 [1.1470556 ]
 [1.2048326 ]
 [1.2626095 ]
 [1.3203862 ]
 [1.3781631 ]
 [1.4359398 ]
 [1.4937167 ]
 [1.5514936 ]
 [1.6092703 ]]
